In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
tf.config.list_physical_devices('GPU')

2023-02-14 17:23:53.436789: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-14 17:23:53.461715: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-14 17:23:53.461764: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [29]:
dataset = pd.read_csv("/home/prp/mercedez_dataset/train.csv")

In [30]:
dataset.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [31]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 378 entries, ID to X385
dtypes: float64(1), int64(369), object(8)
memory usage: 12.1+ MB


In [32]:
dataset.isnull().sum()

ID      0
y       0
X0      0
X1      0
X2      0
       ..
X380    0
X382    0
X383    0
X384    0
X385    0
Length: 378, dtype: int64

In [33]:
x = dataset.drop('y', axis=1)
y = dataset['y']

In [34]:
x.X11.value_counts()

0    4209
Name: X11, dtype: int64

In [35]:
x.columns

Index(['ID', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8', 'X10',
       ...
       'X375', 'X376', 'X377', 'X378', 'X379', 'X380', 'X382', 'X383', 'X384',
       'X385'],
      dtype='object', length=377)

In [36]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 377 entries, ID to X385
dtypes: int64(369), object(8)
memory usage: 12.1+ MB


In [37]:
x = x.drop('ID', axis=1)

In [38]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 376 entries, X0 to X385
dtypes: int64(368), object(8)
memory usage: 12.1+ MB


In [39]:
x.shape

(4209, 376)

In [40]:
x.select_dtypes(include=[object])

,X0,X1,X2,X3,X4,X5,X6,X8
0,k,v,at,a,d,u,j,o
1,k,t,av,e,d,y,l,o
2,az,w,n,c,d,x,j,x
3,az,t,n,f,d,x,l,e
4,az,v,n,f,d,h,d,n
...,...,...,...,...,...,...,...,...
4204,ak,s,as,c,d,aa,d,q
4205,j,o,t,d,d,aa,h,h
4206,ak,v,r,a,d,aa,g,e
4207,al,r,e,f,d,aa,l,u


In [41]:
x = pd.get_dummies(x)

In [42]:
x.shape

(4209, 563)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(x, y, shuffle=True, test_size=0.25, random_state=42)

In [45]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3156, 563), (1053, 563), (3156,), (1053,))

In [46]:
def get_callbacks(model_name):
  callbacks = []
  MC = tf.keras.callbacks.ModelCheckpoint(f'saved_models/{model_name}', monitor='val_loss', save_best_only='True', verbose=1)
  ES = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights='True')
  return [MC, ES]

In [47]:
# Set random seed
tf.random.set_seed(42)

# Build the model (3 layers, 100, 10, 1 units)
model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(100),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1, activation='linear')
], name='mercedes_benz_dataset_model_1')

# Compile the model
model_1.compile(loss='mae',
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=['mae'])

# Fit the model for 200 epochs (same as insurance_model_2)
history_1 = model_1.fit(X_train, y_train, epochs=200, validation_data = (X_test , y_test), callbacks = get_callbacks(model_1.name))

Epoch 1/200


2023-02-14 17:40:08.980634: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 17:40:08.983717: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-14 17:40:08.983818: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-14 17:40:08.983845: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_nod

93/99 [===========================>..] - ETA: 0s - loss: 39.5454 - mae: 39.5454
Epoch 1: val_loss improved from inf to 8.91245, saving model to saved_models/mercedes_benz_dataset_model_1
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(563, 100), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f6344193a30>, 140064797723744), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(100,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f63440d2f70>, 140064797732208), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(100, 10), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f63440f4490>, 140064321084032), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(10,), dtype=tf.float32, name='gradient'), <tensorfl

INFO:tensorflow:Assets written to: saved_models/mercedes_benz_dataset_model_1/assets


INFO:tensorflow:Assets written to: saved_models/mercedes_benz_dataset_model_1/assets


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f63440ae1c0>, 140064321094880), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f63440ae1c0>, 140064321094880), {}).


INFO:tensorflow:Assets written to: saved_models/mercedes_benz_dataset_model_1/assets


INFO:tensorflow:Assets written to: saved_models/mercedes_benz_dataset_model_1/assets


99/99 [==============================] - 1s 8ms/step - loss: 4.7071 - mae: 4.7071 - val_loss: 5.2712 - val_mae: 5.2712
Epoch 16/200
87/99 [=========================>....] - ETA: 0s - loss: 4.7694 - mae: 4.7694
Epoch 16: val_loss did not improve from 5.27117
99/99 [==============================] - 0s 4ms/step - loss: 4.7849 - mae: 4.7849 - val_loss: 5.2825 - val_mae: 5.2825
Epoch 17/200
85/99 [========================>.....] - ETA: 0s - loss: 4.7430 - mae: 4.7430
Epoch 17: val_loss did not improve from 5.27117
99/99 [==============================] - 0s 5ms/step - loss: 4.7462 - mae: 4.7462 - val_loss: 5.4309 - val_mae: 5.4309
Epoch 18/200
92/99 [==========================>...] - ETA: 0s - loss: 4.6944 - mae: 4.6944
Epoch 18: val_loss did not improve from 5.27117
99/99 [==============================] - 0s 4ms/step - loss: 4.6859 - mae: 4.6859 - val_loss: 5.4069 - val_mae: 5.4069
Epoch 19/200
95/99 [===========================>..] - ETA: 0s - loss: 4.7108 - mae: 4.7108
Epoch 19: val_lo